In [1]:
import os
import sys
import time
import pandas as pd
import pyzed.sl as sl
import cv2
import json
from omegaconf import OmegaConf
from tqdm import tqdm
import numpy as np

from vision.misc.help_func import get_repo_dir, write_json, load_json, read_json
from vision.tools.video_wrapper import video_wrapper
from vision.tools.manual_slicer import slice_to_trees

repo_dir = get_repo_dir()
sys.path.append(os.path.join(repo_dir, 'vision', 'detector', 'yolo_x'))

from vision.pipelines.detection_flow import counter_detection
from vision.data.results_collector import ResultsCollector
from vision.tools.translation import translation as T
from vision.tools.image_stitching import plot_2_imgs
from vision.depth.slicer.slicer_flow import post_process
from vision.tools.sensors_alignment import SensorAligner
from vision.tools.camera import is_sturated
from vision.feature_extractor.feature_extractor import create_row_features_fe_pipe
import matplotlib.pyplot as plt
from vision.visualization.drawer import draw_rectangle, draw_text, draw_highlighted_test, get_color
import seaborn as sns
from vision.feature_extractor.image_processing import make_ndvi, ndvi_to_binary

ImportError: cannot import name 'is_sturated' from 'vision.tools.camera' (/home/fruitspec-lab/FruitSpec/Code/roi/fsCounter/vision/tools/camera.py)

In [ ]:
def post_process_slice_df(slice_df):
    """
    Post processes the slices dataframe - if not all frames of the tree are on the json file they are not
        added to the data frame, this function fills in the missing trees with start and end value of -1

    Args:
        slice_df (pd.DataFrame): A dataframe contatining frame_id, tree_id, start, end

    Returns:
        (pd.DataFrame): A post process dataframe
    """
    row_to_add = []
    for tree_id in slice_df["tree_id"].unique():
        temp_df = slice_df[slice_df["tree_id"] == tree_id]
        min_frame, max_frame = temp_df["frame_id"].min(), temp_df["frame_id"].max()
        temp_df_frames = temp_df["frame_id"].values
        for frame_id in range(min_frame, max_frame +1):
            if frame_id not in temp_df_frames:
                row_to_add.append({"frame_id": frame_id, "tree_id": tree_id, "start": -1 ,"end": -1})
    return pd.concat([slice_df, pd.DataFrame.from_records(row_to_add)]).sort_values("frame_id")

In [ ]:
features_df_path = "/media/fruitspec-lab/cam175/APPLECHILE04_test/290323/APPCALIB/APPCALIB_features.csv"
features_df = pd.read_csv(features_df_path)
features_df["customer_name"] = "APPLECHILE04"
features_df["tree_name"] = features_df["name"]
f_path = "/media/fruitspec-lab/cam175/F_dfs/APPLECHILE04_F.csv"
df_f = pd.read_csv(f_path)

In [ ]:
df = features_df
df["full_name"] = df["customer_name"].str.cat(df["block_name"].str.cat(df["tree_name"], sep="_"), sep="_")

In [ ]:
for df in [features_df, df_f]:
    df["full_name"] = df["customer_name"].str.cat(df["block_name"].str.cat(df["tree_name"], sep="_"), sep="_")

In [ ]:
merged_df = df_f.merge(features_df, on = "full_name",suffixes=('', '_y'))
merged_df["cv/F"] = merged_df["cv"]/merged_df["F"]
merged_df["cv/percent_seen"] = merged_df["cv"]/merged_df["percent_seen"]
merged_df["cv*percent_seen"] = merged_df["cv"]*merged_df["percent_seen"]
for i in range(2,6):
    merged_df[f"cv{i}/F"] = merged_df[f"cv{i}"]/merged_df["F"]
    merged_df[f"cv{i}/percent_seen"] = merged_df[f"cv{i}"]/merged_df["percent_seen"]
    merged_df[f"cv{i}*percent_seen"] = merged_df[f"cv{i}"]*merged_df["percent_seen"]

In [ ]:
numer_cols = list(merged_df.select_dtypes(include=np.number).columns)

In [ ]:
pd.concat([merged_df.corr()["cv4/F"].sort_values()[:10], merged_df.corr()["cv4/F"].sort_values()[-10:]])

In [ ]:
# y_col = "cv4/F"
# hue = "row"
# sns.set(font_scale=1)
# # for col in numer_cols:
# for col in ["w_h_ratio", "n_clust_arr_4", "mst_mean_arr"]:
#     if col not in merged_df.columns:
#         continue
#     ax = sns.lmplot(data = merged_df,x = col,y=y_col,hue = hue, scatter_kws={'s':3}, order = 1, height=7, aspect=1,
#                    ci = None)
#     sns.regplot(data = merged_df,x = col,y=y_col, scatter_kws={'s':3}, order = 1,ci=0, ax=ax.axes[0,0],
#                x_ci = 0, color = "black", line_kws={"ls":"--"},scatter=False)
#     plt.show()

In [ ]:
merged_df[[f"cv{i}/F" for i in range(2,6)] + ["cv/F"]].mean()

In [ ]:
merged_df["row"] = merged_df["tree_name"].apply(lambda x: x.split("_")[0])

In [ ]:
def plot_interaction(df,col1,col2, hue = None, title= "", operation="*"):
    df = df.copy()
    if operation == "*":
        col = f"{col1}*{col2}"
        df[col] = df[col1]*df[col2]
    if operation == "/":
        col = f"{col1}/{col2}"
        df[col] = df[col1]/df[col2]
    max_val = np.min(np.max(df[[col, "F"]].values, axis = 0))
    ax = sns.lmplot(data = df, x = col, y="F", hue = hue)
    sns.regplot(data = df, x = col, y="F", scatter_kws={'s':2}, order = 1,ci=0, ax=ax.axes[0,0],
               x_ci = 0, color = "black", line_kws={"ls":"--"},scatter=False)
    plt.plot([0, max_val], [0, max_val], color='grey')
    plt.ylim(0, np.max(df["F"]*1.1))
    plt.title(title)
candiates = ["w_h_ratio", "n_clust_arr_4", "mst_mean_arr"]
plot_interaction(merged_df, "cv4", candiates[0], "row", operation = "/")

In [ ]:
def plot_F_cv(df,min_samp, hue = None, title= "", percent_seen=""):
    if min_samp == 1:
        col = "cv"
    else:
        col = f"cv{i}"
    if percent_seen == "*":
        col = f"{col}*percent_seen"
    if percent_seen == "/":
        col = f"{col}/percent_seen"
    max_val = np.min(np.max(df[[col, "F"]].values, axis = 0))
    ax = sns.lmplot(data = df, x = col, y="F", hue = hue)
    sns.regplot(data = df, x = col, y="F", scatter_kws={'s':2}, order = 1,ci=0, ax=ax.axes[0,0],
               x_ci = 0, color = "black", line_kws={"ls":"--"},scatter=False)
    plt.plot([0, max_val], [0, max_val], color='grey')
    plt.ylim(0, np.max(df["F"]*1.1))
    plt.title(title)
    plt.show()
dropped_outlier_frame = merged_df[~np.isin(merged_df["tree_name"],["R5B_T1", "R5A_T2"])]
dropped_outlier_frame = merged_df
for i in range(1,6)    :
    plot_F_cv(dropped_outlier_frame,i,"row", f"samp{i}")
    plot_F_cv(dropped_outlier_frame,i,"row", f"samp{i} times percent_seen", "*")
    plot_F_cv(dropped_outlier_frame,i,"row", f"samp{i} divide percent_seen", "/")

In [ ]:
merged_df.columns[:50]

In [ ]:
from MHS.scoring import cross_validate_with_mean
from sklearn.linear_model import LinearRegression, PoissonRegressor

class MaxLinearRegressor(LinearRegression):
    def predict(self, X):
        predicted_values = super().predict(X)
        return np.array([max(p, 0) for p in predicted_values])

df = merged_df
groups = df["row"]
only_fulls = True
extra = ["mst_mean_arr", "n_clust_arr_4", "w_h_ratio"]
  
X = df[["cv4"]+ extra]
# X["cv^2"] = X["cv"]**2
y = df["F"]
model = LinearRegression(fit_intercept = False)
# model = MaxLinearRegressor()
y_pred = cross_validate_with_mean(model, X, y, groups=groups, ret_preds=True)
cross_validate_with_mean(model, X, y, groups=groups)

In [ ]:
df = merged_df[~np.isin(merged_df["row"],["R5B", "R5A"])].reset_index()
groups = df["row"]
only_fulls = True

  
X = df[["cv4"]]
# X["cv^2"] = X["cv"]**2
y = df["F"]
model = LinearRegression(fit_intercept = False)
# model = MaxLinearRegressor()
# y_pred = cross_validate_with_mean(model, X, y, groups=groups, ret_preds=True)
cross_validate_with_mean(model, X, y, groups=groups)

In [ ]:
(np.sum(y_pred) - np.sum(y))/np.sum(y)

In [ ]:
cols = ["full_name", "cv", "F", "F/cv", "pred"]
merged_df["pred"] = y_pred
merged_df["err"] = (merged_df["pred"] - merged_df["F"])/merged_df["F"]
merged_df["F/cv"] = merged_df["F"]/merged_df["cv"]
merged_df.sort_values("F/cv")[cols]

In [ ]:
merged_df[merged_df["row"].isin(["R5A"]) ].sort_values("err")[cols + ["err"]]

In [ ]:
plt.hist(merged_df["err"].values)
plt.show()

In [ ]:
merged_df.sort_values("err")[cols + ["err"]]

### config

In [ ]:
row = "/media/fruitspec-lab/cam175/APPLECHILE04/290323/APPCALIB/R2B"
block_name = os.path.basename(os.path.dirname(row))
row_name = os.path.basename(row)
tree_id = 1
rotate = True
t_index=6

### load data

In [ ]:
side = 1 if row.endswith("A") else 2
jai_cam = video_wrapper(os.path.join(row, f"Result_FSI_{side}.mkv"), rotate)
rgb_cam = video_wrapper(os.path.join(row, f"Result_RGB_{side}.mkv"), rotate)
zed_cam = video_wrapper(os.path.join(row, f"ZED_{side}.svo"), 2)
ret, frame = jai_cam.get_frame(1)

In [ ]:
def load_json(filepath):
    """
    this function reads the json file and converts the keys to ints
    :param filepath: path to file
    :return: the converted json
    """
    if not os.path.exists(filepath):
        return {}
    loaded_data = read_json(filepath)
    data = {}
    for k, v in loaded_data.items():
        data[int(k)] = v
    return data

In [ ]:
df_tracks = pd.read_csv(os.path.join(row, "tracks.csv"))
df_slices = pd.read_csv(os.path.join(row, "slices.csv"))
df_coors = pd.read_csv(os.path.join(row, "jai_cors_in_zed.csv"))
jai_zed = load_json(os.path.join(row,"jai_zed.json"))
df_slices = post_process_slice_df(df_slices)

In [ ]:
df_slices

### preprocess

In [ ]:
tree_slice = df_slices[df_slices["tree_id"] == tree_id]
min_frame, max_frame = tree_slice["frame_id"].min(), tree_slice["frame_id"].max()

In [ ]:
tree_tracks = df_tracks[(df_tracks["frame"] >= min_frame) & (df_tracks["frame"] <= max_frame)]
unique_tracks = tree_tracks["track_id"].unique()
new_ids = dict(zip(unique_tracks, range(len(unique_tracks))))
tree_tracks.loc[:, "track_id"] = tree_tracks["track_id"].map(new_ids)

### plotting 

In [ ]:
import numpy as np
import cupy as cp

def num_deno_nan_divide(numerator, denominator, return_numpy=True):
    if isinstance(numerator, type(np.array([]))):
        numerator = cp.array(numerator)
        denominator = cp.array(denominator)
    result = cp.empty(numerator.shape)
    denominator_no_0 = cp.not_equal(denominator, 0)
    result[denominator_no_0] = cp.divide(numerator[denominator_no_0], denominator[denominator_no_0])
    result[1-denominator_no_0] = cp.nan
    if return_numpy:
        return cp.asnumpy(result)
    else:
        return result
    
def num_deno_nan_divide_np(numerator, denominator):
    result = np.full(numerator.shape, np.nan)
    denominator_no_0 = np.abs(denominator) >= 1
    result[denominator_no_0] = numerator[denominator_no_0] / denominator[denominator_no_0]
    result[1-denominator_no_0] = np.nan
    return result

    
def make_ndvi(rgb, nir):
    """
    rgb: rgb image or red channel
    nir: nir image
    return: ndvi image
    """
    if len(rgb.shape) == 3:
        red = rgb[:, :, 0]
    else:
        red = rgb
    ndvi_img = num_deno_nan_divide_np(np.subtract(nir, red), np.add(nir, red))
    np.nan_to_num(ndvi_img, copy=False, nan=0)
    return ndvi_img

def ndvi_to_binary(ndvi, thresh, adpt="", ker=35, const=125):
    """
    turns an image to binary given a threshold
    :param ndvi: ndvi image
    :param thresh: threshold to use for binary transformation
    :param adpt: specify adaptive threshold
    :param ker: kernal for adaptive threshold
    :param const: constant for adaptive threshold
    :return: a binary image
    """
    if adpt == "mean":
        return (
            cv2.adaptiveThreshold(cv2.normalize(ndvi, None, 0, 255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U), 255,
                                  cv2.ADAPTIVE_THRESH_MEAN_C,
                                  cv2.THRESH_BINARY, ker, const))
    if adpt == "gaussian":
        return (
            cv2.adaptiveThreshold(cv2.normalize(ndvi, None, 0, 255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U), 255,
                                  cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                  cv2.THRESH_BINARY, ker, const))
    return cv2.threshold(ndvi, thresh, 1, cv2.THRESH_BINARY)[1]

In [ ]:
def get_min_foli_index(frame):
    red, nir = frame[:,:,2].astype(np.float32), frame[:,:,0].astype(np.float32)
    ndvi = make_ndvi(red, nir)
    binary_ndvi = 1-ndvi_to_binary(ndvi, thresh=-0.1)
#     plt.imshow(binary_ndvi)
#     plt.show()
    np.sum(binary_ndvi, axis = 1)
    return np.max(np.where(np.sum(binary_ndvi, axis = 1)>250))

# get_min_foli_index()

In [ ]:
def thresh_channels(frame, channels_thresh = {0:(0, 100), 1:(25,255), 2:(0,150)},
                   gree_minus_red_thres = (0,40)):
    frame = frame.copy()
    for channel, thresholds in channels_thresh.items():
        chan = frame[:,:,channel]
        valids = np.all([chan > thresholds[0], chan < thresholds[1]], axis = 0)
        frame[(1-valids).astype(np.bool)] = 0
    if np.sum(gree_minus_red_thres) > 0:
        valids = np.all([frame[:,:,1] - frame[:,:,0]<gree_minus_red_thres[1],
                   frame[:,:,1] - frame[:,:,0]>gree_minus_red_thres[0]], axis = 0)
        frame[(1-valids).astype(np.bool)] = 0
    return frame[:,:,1] > 0


def get_percent_seen(zed_frame, jai_frame, jai_regb_frame, coors):
    x1,y1,x2,y2 = coors
    y2 = zed_frame.shape[0]
    zed_threshed = thresh_channels(zed_frame, channels_thresh = {0:(0, 100), 1:(25,255), 2:(0,150)})    
    zed_threshed[y1:y2, x1:x2] = 0
    zed_threshed_cut = zed_threshed[:, x1:x2]
    return 1-np.mean(np.mean(zed_threshed_cut, axis =0))


In [ ]:
f_id = 2700
zed_f_id = jai_zed[f_id] 
ret, jai_frame = jai_cam.get_frame(f_id)
ret, rgb_frame = rgb_cam.get_frame(f_id)
zed_frame, point_cloud = zed_cam.get_zed(zed_f_id, exclude_depth=True)
jai_frame = jai_frame[:,:,::-1]
zed_frame = zed_frame[:,:,::-1]
plot_2_imgs(jai_frame, zed_frame)

In [ ]:
x1,y1,x2,y2 = df_coors[df_coors["frame"] == f_id][["x1","y1", "x2","y2"]].values[0]

In [ ]:
jai_in_zed = zed_frame[y1:y2, x1:x2]
pc_jai_in_zed = point_cloud[y1:y2, x1:x2]
plot_2_imgs(jai_frame, jai_in_zed)

In [ ]:
ndvi = make_ndvi(rgb_frame.astype(np.float32), jai_frame[:,:,0].astype(np.float32))
plot_2_imgs(ndvi, jai_frame)

In [ ]:
binary_ndvi = ndvi_to_binary(ndvi, 0.05)
plot_2_imgs(ndvi, binary_ndvi)

In [ ]:
zed_threshed = thresh_channels(zed_frame)    
plot_2_imgs(zed_frame, zed_threshed)

In [ ]:
zed_threshed[y1:1920, x1:x2] = 0
zed_threshed_cut = zed_threshed[:, x1:x2]
plt.imshow(zed_threshed_cut)
plt.show()

In [ ]:
zed_in_jai_full = np.zeros_like(zed_threshed)
zed_in_jai_full[y1:y2, x1:x2] = cv2.resize(binary_ndvi, (x2-x1,y2-y1))
diff_img = zed_threshed.astype(np.int) - zed_in_jai_full.astype(np.int) > 0
plot_2_imgs(zed_in_jai_full, diff_img)

In [ ]:
s